<h2> RAMP on qualitative and quantitative non-invasive monitoring of anti-cancer drugs </h2>

<i> M2 Data Science - Chia-Man Hung - chia-man.hung@polytechnique.edu </i>

## Introduction

This document aims at presenting my approaches to Data Camp 2016. http://www.ramp.studio/ <br />
The code source can be found at on my github. https://github.com/ascane/data-camp <br />
I will only include code that I wrote myself (not from the starting kit). Since I will not copy and load the dataset, the code below cannot be run directly from this file. Please refer to the code source in case you would like to run it yourself.

## Data Exploration

Besides the plots provided by the starting kit, I tried to visualize the data myself. Below is a plot of data with the same molecule and the same concentration.

In [ ]:
for mol in np.unique(molecule):
    for c in np.unique(concentration):
        if len(X[(concentration == c) & (molecule == mol), :]) > 0:
            plt.figure()
            plt.plot(X[(concentration == c) & (molecule == mol), :].T)
            plt.title(mol + " - %s - %s samples." % (c, X[(concentration == c) & (molecule == mol), :].shape[0]))

## Building my own model: Gaussian distributions

This is the most original part of this document. You may like to take a close look at it.

### Principle
The idea behind this model is to group data with the same molecule and the same concentration and estimate the probability distribution of each group. More specifically, in the feature_extractor_clf, we pass the group as a feature to the classifier. In the classifier, for each group, we compute a Gaussian distribution (its mean and its covariance) based on the data in a large dimension (number of frequencies chosen).

Our goal is that given a test data, we want to predict the probability of having a certain molecule. We do it by computing the probability of belonging to each group based on the estimated distribution of each group. 

$\forall mol \in molecules, p(mol | test data)$ is proportional to $p(test data | mol) p(mol) = \sum_{c} p(test data | mol , c) p(mol, c)$

We estimate $p(test data | mol , c)$ by the precomputed Gaussian distribution and the $p(mol, c)$ by the proportion of the train data in each group.

### Numerical issues
In fact, the covariance matrices computed this way have no chance to be invertible because the number of frequencies chosen is much greater than the number of train data we have. However, to compute the probability, we need it to be invertible. A classical trick consists of adding an identity matrix. Another issue is then how much we add. If we add too little, some of the covariance matrices are still not invertible; if we add to much, they are almost identity. We started by tweaking its weight and observing the numerical values. Later, we found that this could be optimized by quadratic programming with linear constraints (cvxopt). Theoretical details are provided in the Reference section.

The number of frequencies chosen is another parameter to tune. In the local case, there are about 1800 different frequencies. Such a big number is infeasible for the matrix computation.

feature_extractor_clf.py

In [ ]:
import numpy as np
import pandas as pd

class FeatureExtractorClf(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_df):
        self.X_df_fit = X_df
        self.y_df_fit = y_df
        pass
    
    def transform(self, X_df):
        valueArray = X_df['spectra'].values
        for i in range(len(valueArray)):
            valueArray[i] = [valueArray[i][ind] for ind in range(len(valueArray[i])) if ind % 20 == 0]
        if X_df is self.X_df_fit:
            return zip(valueArray, self.y_df_fit.values)
        return zip(valueArray, [None] * len(valueArray))

classifier.py

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
import numpy as np
from scipy.stats import multivariate_normal

class Classifier(BaseEstimator):
    def __init__(self):
        self.group_m = []
        self.group_c = []
        self.group_mean = []
        self.group_cov = []
        self.group_pdf = []
        self.group_size = []
        self.group_n = 0
        self.group_total_size = 0;
        pass

    def fit(self, X, y):
        epsilon = 3.5 * 1e-7
        n = len(X)
        self.group_total_size = n
        X_values = [x[0] for x in X]
        X_molecules = [x[1][0] for x in X]
        X_concentrations = [x[1][1] for x in X]
        molecules = np.unique(X_molecules)
        concentrations = np.unique(X_concentrations)
        for m in molecules:
            for c in concentrations:
                mc_values = [X_values[i] for i in range(n) if X_molecules[i] == m and X_concentrations[i] == c]
                if len(mc_values) > 0:
                    self.group_m.append(m)
                    self.group_c.append(c)
                    self.group_mean.append([0] * len(mc_values[0]))
                    for i in range(len(mc_values[0])):
                        self.group_mean[self.group_n][i] = np.mean([x[i] for x in mc_values])
                    self.group_cov.append(np.cov(np.transpose(mc_values)))
                    for i in range(len(mc_values[0])):
                        for j in range(len(mc_values[0])):
                            self.group_cov[self.group_n][i][j] += epsilon
                        self.group_cov[self.group_n][i][i] += epsilon
                    self.group_pdf.append(multivariate_normal(mean=self.group_mean[self.group_n], cov=self.group_cov[self.group_n]))
                    self.group_size.append(len(mc_values))
                    self.group_n += 1
                    
                    plt.plot(self.group_mean[self.group_n - 1], label="%s %s" % (m, c))
                

    def predict(self, X):
        pass

    def predict_proba(self, X):
        mol = ['A', 'B', 'Q', 'R']
        result = []
        for i in range(len(X)):
            result.append([0,0,0,0])
            m = -1
            for j in range(self.group_n):
                for k in range(len(mol)):
                    if self.group_m[j] == mol[k]:
                        m = k
                result[i][m] += self.group_pdf[j].pdf(X[i][0]) * self.group_size[j] / float(self.group_total_size)
            prob_sum = sum(result[i])
            for j in range(4):
                result[i][j] /= prob_sum
            
        return np.array(result)

### Local results

Below is the result I got running it locally.

error = 0.12 <br />
classification report: <br />
              precision    recall  f1-score   support

          A       0.94      0.97      0.95        63
          B       0.94      0.76      0.84        45
          Q       0.90      0.90      0.90        40
          R       0.76      0.87      0.81        52

avg / total       0.89      0.88      0.88       200 <br />

confusion matrix: <br />
 [[61  0  0  2] <br />
 [ 1 34  1  9] <br />
 [ 1  0 36  3] <br />
 [ 2  2  3 45]] <br />
 
 Note that the two parameters mentioned above (weight of the identity matrix, number of frequencies chosen) can still be tuned to obtain better results. This error is comparable to the untuned RandomForestClassifer provided in the starting kit.

### Submission on the server

I tried to submit this even though it does not seem to perform much better than the RandomForestClassifier provided by the starting kit. However, due to a descrepency between the local test and the server, my submission failed. (In the user test fe_clf.fit() received the pandas table y_df, whereas on the server we sent only the molecule types in a 1D numpy array.) After the starting kit got updated, I had only one submission left. Therefore, I did not have a chance to submit it on the server.

I wanted to implement the same idea in the regressor but I did not continue on this.

### Reference
A well-conditioned estimator for large-dimensional covariance matrices, O. Ledoit, M. Wolf, February 2001. <br />
http://perso.ens-lyon.fr/patrick.flandrin/LedoitWolf_JMA2004.pdf

## Cross validation (Hyperparameters tuning)

I started by using GridSearchCV in sklearn.model_selection, but the results were only given at the end of the training. It was not very convenient and I could not estimate when it would terminate. My workaround using lots of for loops and printing all the results out as seen below. There are a lot of parameters to tune and it is impossible to run them all at a time. First, I selected some of them and set the step in range to be quite big to to have a vague idea. Then, I reduced the range and decreased the step sizes to have more precise values.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.svm import SVC

class Classifier(BaseEstimator):
    def __init__(self, n_components, C, degree, coef0, gamma, kernel):
        self.n_components = n_components
        self.C = C
        self.degree = degree
        self.coef0 = coef0
        self.gamma = gamma
        self.kernel = kernel
        self.clf = Pipeline([
            ('pca', PCA(n_components=self.n_components)), 
            ('svc', SVC(C=self.C, kernel=self.kernel, degree=self.degree, gamma=self.gamma, coef0=self.coef0, shrinking=True, probability=True, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=None, random_state=42))
        ])

    def fit(self, X, y):
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [ ]:
def frange(x, y, jump):
    while x < y:
        yield x
        x += jump

In [ ]:
import numpy as np
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

labels = np.array(['A', 'B', 'Q', 'R'])

def train_test_model_clf(X_df, y_df, skf_is, FeatureExtractor, Classifier, n_components, C, degree, coef0, gamma, kernel):
    train_is, test_is = skf_is
    X_train_df = X_df.iloc[train_is].copy()                                  
    y_train_df = y_df.iloc[train_is].copy()
    y_train_clf = y_train_df['molecule'].values
    X_test_df = X_df.iloc[test_is].copy()                                    
    y_test_df = y_df.iloc[test_is].copy() 
    y_test_clf = y_test_df['molecule'].values 
    # Feature extraction
    fe_clf = FeatureExtractor()
    fe_clf.fit(X_train_df, y_train_df)
    X_train_array_clf = fe_clf.transform(X_train_df)
    X_test_array_clf = fe_clf.transform(X_test_df)
    # Train
    clf = Classifier(n_components, C, degree, coef0, gamma, kernel)
    clf.fit(X_train_array_clf, y_train_clf)
    # Test 
    y_proba_clf = clf.predict_proba(X_test_array_clf)                        
    y_pred_clf = labels[np.argmax(y_proba_clf, axis=1)]                      
    error = 1 - accuracy_score(y_test_clf, y_pred_clf)
    print('n_components = %s - C = %s - degree = %s - coef0 = %s - gamma = %s - kernel = %s' % (n_components, C, degree, coef0, gamma, kernel))
    print('error = %s' % error)
    return error


kernels = ['linear', 'poly', 'rbf', 'sigmoid']

best_n_components, best_C, best_degree, best_coef, best_gamma, best_kernel, best_error = -1, -1, -1, -1, -1, '', 100
for n_components in range(8, 12, 1):
    for C in frange(8.3, 8.4, 0.1):
        for degree in range(10, 11, 2):
            for coef0 in frange(3.4, 3.5, 0.1):
                for gamma in frange(0.05, 0.06, 0.01):
                    for kernel in kernels:
                        error = []
                        for t in range(5):
                            skf = ShuffleSplit(n_splits=2, test_size=0.2, random_state=57)  
                            skf_is = list(skf.split(X_df))[0]
                            error.append(train_test_model_clf(X_df, y_df, skf_is, FeatureExtractorClf, Classifier, n_components, C, degree, coef0, gamma, kernel))
                        if np.mean(error) < best_error:
                            best_error = np.mean(error)
                            best_n_components, best_C, best_degree, best_coef0, best_gamma, best_kernel = n_components, C, degree, coef0, gamma, kernel

print('best_n_components = %s' % best_n_components)
print('best_C = %s' % best_C)
print('best_degree = %s' % best_degree)
print('best_coef0 = %s' % best_coef0)
print('best_gamma = %s' % best_gamma)
print('best_kernel = %s' % best_kernel)
print('best_error = %s' % best_error)

### Local results

Below are the intermediate results found running the previous code. The unspecified parameters are set to default. <br />

best_n_components = 100, best_C = 8, best_degree = 10, best_coef0 = 3, best_kernel = poly, best_error = 0.038 <br />
best_n_components = 100, best_C = 7, best_degree = 10, best_coef0 = 3, best_kernel = poly, best_error = 0.039 <br />
best_n_components = 100, best_C = 8.3, best_degree = 10, best_coef0 = 3.5, best_kernel = poly, best_error = 0.033 <br />
best_n_components = 100, best_C = 8.1, best_degree = 10, best_coef0 = 3.4, best_kernel = poly, best_error = 0.037 <br />
best_n_components = 100, best_C = 8.3, best_degree = 10, best_coef0 = 3.4, best_gamma = 0.1, best_kernel = poly, best_error = 0.033 <br />
best_n_components = 100, best_C = 8.3, best_degree = 10, best_coef0 = 3.4, best_gamma = 0.05, best_kernel = poly, best_error = 0.031 <br />
best_n_components = 100, best_C = 8.3, best_degree = 10, best_coef0 = 3.4, best_gamma = 0.02, best_kernel = poly, best_error = 0.035 <br />
best_n_components = 100, best_C = 8.3, best_degree = 10, best_coef0 = 3.4, best_gamma = 0.04, best_kernel = poly, best_error = 0.03 <br />
best_n_components = 100, best_C = 8.3, best_degree = 10, best_coef0 = 3.4, best_gamma = 0.05, best_kernel = poly, best_error = 0.034 <br />
best_n_components = 10, best_C = 8.3, best_degree = 10, best_coef0 = 3.4, best_gamma = 0.05, best_kernel = poly, best_error = 0.02 <br />

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor                           
from sklearn.decomposition import PCA                                            
from sklearn.pipeline import Pipeline                                            
from sklearn.base import BaseEstimator                                           
import numpy as np                                                               
                           
class Regressor(BaseEstimator):                                                  
    def __init__(self, n_components, n_estimators, learning_rate, loss, subsample, min_samples_split, min_samples_leaf, min_weight_fraction_leaf, max_depth, min_impurity_split, alpha):                                                       
        self.n_components = n_components                                         
        self.n_estimators = n_estimators                                             
        self.learning_rate = learning_rate
        self.loss = loss
        self.subsample = subsample
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_weight_fraction_leaf = min_weight_fraction_leaf
        self.max_depth = max_depth
        self.min_impurity_split = min_impurity_split
        self.alpha = alpha
        self.list_molecule = ['A', 'B', 'Q', 'R']                                
        self.dict_reg = {}                                                       
        for mol in self.list_molecule:                                           
            self.dict_reg[mol] = Pipeline([                                      
                ('pca', PCA(n_components=self.n_components)),                    
                ('reg', GradientBoostingRegressor(
                    n_estimators=self.n_estimators,                              
                    learning_rate=self.learning_rate, 
                    loss=self.loss, # 'ls'
                    subsample=self.subsample, #1.0
                    criterion='friedman_mse',
                    min_samples_split=self.min_samples_split, # 2
                    min_samples_leaf=self.min_samples_leaf, # 1
                    min_weight_fraction_leaf=self.min_weight_fraction_leaf, # 0.0
                    max_depth=self.max_depth, # 3
                    min_impurity_split=self.min_impurity_split, # 1e-07
                    init=None,
                    max_features='sqrt',
                    alpha=self.alpha, # 0.9
                    max_leaf_nodes=None,
                    warm_start=False,
                    random_state=42))                                            
            ])                                                                   
                                                                                 
    def fit(self, X, y):                                                         
        for i, mol in enumerate(self.list_molecule):                             
            ind_mol = np.where(np.argmax(X[:, -4:], axis=1) == i)[0]             
            XX_mol = X[ind_mol]                                                  
            y_mol = y[ind_mol].astype(float)                                     
            self.dict_reg[mol].fit(XX_mol, np.log(y_mol))                        
                                                                                 
    def predict(self, X):                                                        
        y_pred = np.zeros(X.shape[0])                                            
        for i, mol in enumerate(self.list_molecule):                             
            ind_mol = np.where(np.argmax(X[:, -4:], axis=1) == i)[0]             
            XX_mol = X[ind_mol].astype(float)                                    
            y_pred[ind_mol] = np.exp(self.dict_reg[mol].predict(XX_mol))         
        return y_pred

In [ ]:
def mare_score(y_true, y_pred):                                                  
    return np.mean(np.abs((y_true - y_pred) / y_true)) 

def train_test_model(X_df, y_df, skf_is, FeatureExtractorClf, Classifier, FeatureExtractorReg, RegressorClassifier, n_components_2, n_estimators, learning_rate, loss, subsample, min_samples_split, min_samples_leaf, min_weight_fraction_leaf, max_depth, min_impurity_split, alpha, n_components=10, C=8.3, degree=10, coef0=3.4, gamma=0.05, kernel='poly'):
    train_is, test_is = skf_is
    X_train_df = X_df.iloc[train_is].copy()                                  
    y_train_df = y_df.iloc[train_is].copy()                                  
    X_test_df = X_df.iloc[test_is].copy()                                    
    y_test_df = y_df.iloc[test_is].copy()                                    
    y_train_clf = y_train_df['molecule'].values                              
    y_train_reg = y_train_df['concentration'].values                         
    y_test_clf = y_test_df['molecule'].values                                
    y_test_reg = y_test_df['concentration'].values                           

    # Classification
    fe_clf = FeatureExtractorClf()                     
    fe_clf.fit(X_train_df, y_train_df)                                       
    X_train_array_clf = fe_clf.transform(X_train_df)                         
    X_test_array_clf = fe_clf.transform(X_test_df)                           
                                                                                 
    clf = Classifier(n_components, C, degree, coef0, gamma, kernel)                                          
    clf.fit(X_train_array_clf, y_train_clf)                                  
    y_proba_clf = clf.predict_proba(X_test_array_clf)                        
    y_pred_clf = labels[np.argmax(y_proba_clf, axis=1)]                      
    error = 1 - accuracy_score(y_test_clf, y_pred_clf)                       
    
    # Regression
    fe_reg = FeatureExtractorReg()                     
    for i, label in enumerate(labels):
        # For training, we use 
        X_train_df.loc[:, label] = (y_train_df['molecule'] == label)         
        X_test_df.loc[:, label] = y_proba_clf[:, i]                          
    fe_reg.fit(X_train_df, y_train_reg)                                      
    X_train_array_reg = fe_reg.transform(X_train_df)                         
    X_test_array_reg = fe_reg.transform(X_test_df)                           
                                                                                 
    reg = Regressor(n_components_2, n_estimators, learning_rate, loss, subsample, min_samples_split, min_samples_leaf, min_weight_fraction_leaf, max_depth, min_impurity_split, alpha)                                              
    reg.fit(X_train_array_reg, y_train_reg)                               
    y_pred_reg = reg.predict(X_test_array_reg)
    mare = mare_score(y_test_reg, y_pred_reg)
    print('n_components_2 = %s - n_estimator = %s - learning_rate = %s - loss = %s - subsample = %s - min_samples_split = %s - min_samples_leaf = %s - min_weight_fraction_leaf = %s - max_depth = %s - min_impurity_split = %s - alpha = %s' % (n_components_2, n_estimators, learning_rate, loss, subsample, min_samples_split, min_samples_leaf, min_weight_fraction_leaf, max_depth, min_impurity_split, alpha))
    print('mare = ', mare)                
    print('combined error = ', 2. / 3 * error + 1. / 3 * mare)
    return mare


losses = ['ls', 'lad', 'huber', 'quantile']

best_n_components_2, best_n_estimators, best_learning_rate, best_loss, best_subsample, best_min_samples_split, best_min_samples_leaf, best_min_weight_fraction_leaf, best_max_depth, best_min_impurity_split, best_alpha, best_error = -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 100
for n_components_2 in range(10, 15, 5):
    for n_estimators in range(100, 1500, 200):
        for learning_rate in frange(0.05, 0.25, 0.05):
            for loss in losses:
                for subsample in frange(0.9, 1, 0.2):
                    for min_samples_split in range(2, 3, 1):
                        for min_samples_leaf in range(1, 2, 1):
                            for min_weight_fraction_leaf in frange(0, 0.1, 0.1):
                                for max_depth in range(3, 5, 2):
                                    for min_impurity_split in frange(1e-7, 2e-7, 1e-7):
                                        for alpha in frange(0.9, 1.0, 0.1):
                                            error = []
                                            for t in range(5):
                                                skf = ShuffleSplit(n_splits=2, test_size=0.2, random_state=57)  
                                                skf_is = list(skf.split(X_df))[0]
                                                error.append(train_test_model(X_df, y_df, skf_is, FeatureExtractorClf, Classifier, FeatureExtractorReg, Regressor, n_components_2, n_estimators, learning_rate, loss, subsample, min_samples_split, min_samples_leaf, min_weight_fraction_leaf, max_depth, min_impurity_split, alpha))
                                            if np.mean(error) < best_error:
                                                best_error = np.mean(error)
                                                best_n_components_2, best_n_estimators, best_learning_rate, best_loss, best_subsample, best_min_samples_split, best_min_samples_leaf, best_min_weight_fraction_leaf, best_max_depth, best_min_impurity_split, best_alpha = n_components_2, n_estimators, learning_rate, loss, subsample, min_samples_split, min_samples_leaf, min_weight_fraction_leaf, max_depth, min_impurity_split, alpha
print('best_n_components_2 = %s - best_n_estimator = %s - best_learning_rate = %s - best_loss = %s - best_subsample = %s - best_min_samples_split = %s - best_min_samples_leaf = %s - best_min_weight_fraction_leaf = %s - best_max_depth = %s - best_min_impurity_split = %s - best_alpha = %s' % (best_n_components_2, best_n_estimators, best_learning_rate, loss, subsample, min_samples_split, min_samples_leaf, min_weight_fraction_leaf, max_depth, min_impurity_split, alpha))
print('best_mare = %s' % best_error)

### Local results

The unspecified parameters are set to default.

best_n_components_2 = 10 - best_n_estimator = 110 - best_learning_rate = 0.2 - best_mare = 0.15797804292

best_n_components_2 = 10 - best_n_estimator = 110 - best_learning_rate = 0.15 - best_mare = 0.159668444652

best_n_components_2 = 10 - best_n_estimator = 300 - best_learning_rate = 0.2 - best_mare = 0.158506996752

best_n_components_2 = 10 - best_n_estimator = 300 - best_learning_rate = 0.2 - best_loss = huber - best_subsample = 0.9 - best_min_samples_split = 2 - best_min_samples_leaf = 1 - best_min_weight_fraction_leaf = 0 - best_max_depth = 3 - best_min_impurity_split = 1e-07 - best_alpha = 0.9 - best_mare = 0.152787475799

best_n_components_2 = 10 - best_n_estimator = 300 - best_learning_rate = 0.2 - best_loss = huber - best_subsample = 0.9 - best_min_samples_split = 200 - best_min_samples_leaf = 1 - best_min_weight_fraction_leaf = 0 - best_max_depth = 7 - best_min_impurity_split = 1e-07 - best_alpha = 0.9 - best_mare = 0.157893457824

best_n_components_2 = 10 - best_n_estimator = 300 - best_learning_rate = 0.2 - best_loss = huber - best_subsample = 0.9 - best_min_samples_split = 10 - best_min_samples_leaf = 4 - best_min_weight_fraction_leaf = 0 - best_max_depth = 7 - best_min_impurity_split = 1e-07 - best_alpha = 0.9 - best_mare = 0.169251068628

best_n_components_2 = 10 - best_n_estimator = 130 - best_learning_rate = 0.2 - best_loss = huber - best_subsample = 0.9 - best_min_samples_split = 2 - best_min_samples_leaf = 1 - best_min_weight_fraction_leaf = 0 - best_max_depth = 3 - best_min_impurity_split = 1e-07 - best_alpha = 0.9 - best_mare = 0.153183626478

best_n_components_2 = 10 - best_n_estimator = 140 - best_learning_rate = 0.2 - best_loss = ls - best_subsample = 0.9 - best_min_samples_split = 2 - best_min_samples_leaf = 1 - best_min_weight_fraction_leaf = 0 - best_max_depth = 3 - best_min_impurity_split = 1e-07 - best_alpha = 0.9 - best_mare = 0.151323405277

best_n_components_2 = 10 - best_n_estimator = 100 - best_learning_rate = 0.2 - best_loss = ls - best_subsample = 0.9 - best_min_samples_split = 2 - best_min_samples_leaf = 1 - best_min_weight_fraction_leaf = 0 - best_max_depth = 3 - best_min_impurity_split = 1e-07 - best_alpha = 0.9 - best_mare = 0.152335430693

best_n_components_2 = 10 - best_n_estimator = 700 - best_learning_rate = 0.15 - best_loss = ls - best_subsample = 0.9 - best_min_samples_split = 2 - best_min_samples_leaf = 1 - best_min_weight_fraction_leaf = 0 - best_max_depth = 3 - best_min_impurity_split = 1e-07 - best_alpha = 0.9 - best_mare = 0.151876105299

### Reference
Cross Validation & Ensembling <br />
http://datalab-lsml.appspot.com/lectures/08_CV_Ensembling.html <br />
Parameter tuning Gradient boosting <br />
https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

## Final Submission

After trying several models in sklearn, the SVC turns out to be the best for the classifier and the GradientBoostingRegressor turns out to be the best for the regressor. I didn't manage to add other features that improve the results, so the feature extractors stay the same as in the starting kit. Hyperparameters are tuned according to the best results found in the previous section.

classifier.py

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.svm import SVC

class Classifier(BaseEstimator):
    def __init__(self):
        self.n_components = 10
        self.clf = Pipeline([
            ('pca', PCA(n_components=self.n_components)), 
            ('svc', SVC(C=8.3, kernel='poly', degree=10, gamma=0.05, coef0=3.4, shrinking=True, probability=True, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=None, random_state=None))
        ])

    def fit(self, X, y):
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)

regressor.py

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor                           
from sklearn.decomposition import PCA                                            
from sklearn.pipeline import Pipeline                                            
from sklearn.base import BaseEstimator                                           
import numpy as np                                                               
                                                                                 
                                                                                 
class Regressor(BaseEstimator):                                                  
    def __init__(self):                                                          
        self.n_components = 10                                        
        self.n_estimators = 140                                              
        self.learning_rate = 0.2
        self.loss = 'ls'
        self.subsample = 0.9
        self.min_samples_split = 2
        self.min_samples_leaf = 1
        self.min_weight_fraction_leaf = 0.0
        self.max_depth = 3
        self.min_impurity_split = 1e-07
        self.alpha = 0.9
        
        self.list_molecule = ['A', 'B', 'Q', 'R']                                
        self.dict_reg = {}                                                       
        for mol in self.list_molecule:                                           
            self.dict_reg[mol] = Pipeline([                                      
                ('pca', PCA(n_components=self.n_components)),                    
                ('reg', GradientBoostingRegressor(
                    n_estimators=self.n_estimators,                              
                    learning_rate=self.learning_rate, 
                    loss=self.loss, # 'ls'
                    subsample=self.subsample, #1.0
                    criterion='friedman_mse',
                    min_samples_split=self.min_samples_split, # 2
                    min_samples_leaf=self.min_samples_leaf, # 1
                    min_weight_fraction_leaf=self.min_weight_fraction_leaf, # 0.0
                    max_depth=self.max_depth, # 3
                    min_impurity_split=self.min_impurity_split, # 1e-07
                    init=None,
                    max_features=None,
                    alpha=self.alpha, # 0.9
                    max_leaf_nodes=None,
                    warm_start=False,
                    random_state=42))                                            
            ])                                                                   
                                                                                 
    def fit(self, X, y):                                                         
        for i, mol in enumerate(self.list_molecule):                             
            ind_mol = np.where(np.argmax(X[:, -4:], axis=1) == i)[0]             
            XX_mol = X[ind_mol]                                                  
            y_mol = y[ind_mol].astype(float)                                     
            self.dict_reg[mol].fit(XX_mol, np.log(y_mol))                        
                                                                                 
    def predict(self, X):                                                        
        y_pred = np.zeros(X.shape[0])                                            
        for i, mol in enumerate(self.list_molecule):                             
            ind_mol = np.where(np.argmax(X[:, -4:], axis=1) == i)[0]             
            XX_mol = X[ind_mol].astype(float)                                    
            y_pred[ind_mol] = np.exp(self.dict_reg[mol].predict(XX_mol))         
        return y_pred  

### Local results

Results from !python user_test_submission.py <br />

Reading file ... <br />
Training file ... <br />
-------------------------- <br />
error = 0.02 <br />
('mare = ', 0.16059234361494051) <br />
('combined error = ', 0.066864114538313521) <br />
-------------------------- <br />
error = 0.04 <br />
('mare = ', 0.1562354813434049) <br />
('combined error = ', 0.078745160447801651) <br />

### Submission on the server

Combined error = 0.074 <br />
error = 0.032 <br />
mare = 0.157 <br />